In [88]:
import pandas as pd
import numpy as np
from scipy.stats import entropy

from pgmpy.models import BayesianModel
from pgmpy.estimators import HillClimbSearch, K2Score, BicScore
from pgmpy.sampling.Sampling import BayesianModelSampling
from pgmpy.factors.discrete.CPD import TabularCPD

def JSD(P, Q):
    # a divergence measure
    _P = P / np.linalg.norm(P, ord=1)
    _Q = Q / np.linalg.norm(Q, ord=1)
    _M = 0.5 * (_P + _Q)
    return 0.5 * (entropy(_P, _M) + entropy(_Q, _M))

def total_variation_distance(P, Q):
    _P = P / np.linalg.norm(P, ord=1)
    _Q = Q / np.linalg.norm(Q, ord=1)
    return np.abs(_P-_Q).max()

## Datasets Setup

In [2]:
data = pd.DataFrame.from_csv('crash_sample_2018.csv', index_col=None)
data.set_index('Unnamed: 0', inplace=True)

nodes = list(data.columns)
cat_to_num = {'AvgSpeed': {'low': 0, 'high': 1}, \
              'Country': {'US': 0, 'UK': 1, 'Europe': 2}, \
              'DangerLvl': {'low': 0, 'high': 1}, \
              'NoAccidents': {'low': 0, 'medium': 1, 'high': 2}, \
              'NoFatalities': {'low': 0, 'medium': 1, 'high': 2}, \
              'NoJourneys': {'low': 0, 'medium': 1, 'high': 2}, \
              'PoliceActivity': {'regular': 0, 'increased': 1}, \
              'RoadCond': {'bad': 0, 'good': 1}, \
              'Season': {'winter': 0, 'spring': 1, 'summer': 2, 'fall': 3}, \
              'Weather': {'bad': 0, 'good': 1}, \
              'Weekend': {'working': 0, 'weekend': 1, 'holiday': 2},
              }

num_to_cat = {}
for k1, v1 in cat_to_num.items():
    num_to_cat[k1] = {v2: k2 for k2, v2 in v1.items()}


data.replace(cat_to_num, inplace=True)

incomplete_mask = data.isnull().any(axis=1)
incomplete_data = data[incomplete_mask].copy()
complete_data = data[~incomplete_mask]

training_mask = np.random.rand(len(complete_data)) < 0.8
training_data = complete_data[training_mask]
test_data = complete_data[~training_mask]


## Baseline Model Construction

In [3]:
def get_model():
    edges = [
        ("Season", "Weather"),
        ("Weather", "RoadCond"),
        ("RoadCond", "AvgSpeed"),
        ("Country", "AvgSpeed"),
        ("Weekend", "NoJourneys"),
        ("NoJourneys", "AvgSpeed"),
        ("PoliceActivity", "DangerLvl"),
        ("RoadCond", "DangerLvl"),
        ("AvgSpeed", "DangerLvl"),
        ("DangerLvl", "NoAccidents"),
        ("NoAccidents", "NoFatalities"),
    ]
    model = BayesianModel(edges)
    return model

In [4]:
def check_independence(model, group1, group2, observed=[]):
    for node1 in group1:
        for node2 in group2:
            if model.is_active_trail(node1, node2, observed):
                return False
    return True

model = get_model()
print(check_independence(model, ["Season"], ["NoFatalities"],["NoJourneys"]))
print(check_independence(model, ["Weather"], ["NoAccidents"],["RoadCond"]))
print(check_independence(model, ["Season"], ["Weekend"],["NoAccidents"]))

False
True
False


## CPT Learning (EM + MLE)

In [9]:
from pgmpy.estimators import MaximumLikelihoodEstimator
from pgmpy.inference import VariableElimination

"""
Warning: runs unsatisfactorily slow. 
"""

def regenerate_incomplete_data(row, infer=None):
    missing_column_mask = row.isnull()
    incomplete_row = row[missing_column_mask]
    complete_row = row[~missing_column_mask]
    
    missing_nodes = list(incomplete_row.to_dict().keys())
    evidence = complete_row.astype(int).to_dict()

    if len(missing_nodes) == 0:
        return row
    
    prediction = infer.map_query(missing_nodes, evidence=evidence)
    evidence.update(prediction)
    return pd.Series(evidence)

def e(incomplete_data, model):
    infer = VariableElimination(model)
    return incomplete_data.apply(regenerate_incomplete_data, axis=1, infer=infer)

def m(training_data):
    model = get_model()
    mle = MaximumLikelihoodEstimator(model, training_data)
    model.add_cpds(*mle.get_parameters())
    return model

model = m(training_data)

for i in range(10):
    print("Iteration %d: " % (i+1), end="")
    completed_data = e(incomplete_data, model)
    model = m(pd.concat([training_data, completed_data]))
    print("Completed")
    
full_completed_data = pd.concat([complete_data, completed_data])

Iteration 1: Completed
Iteration 2: Completed
Iteration 3: Completed
Iteration 4: Completed
Iteration 5: Completed
Iteration 6: Completed
Iteration 7: Completed
Iteration 8: Completed
Iteration 9: Completed
Iteration 10: Completed


In [91]:
print(model.get_cpds("AvgSpeed"))

╒═══════════════╤═════════════════════╤═════════════════════╤═════════════════════╤════════════════════╤═════════════════════╤═════════════════════╤═════════════════════╤════════════════════╤═════════════════════╤════════════════════╤═════════════════════╤═════════════════════╕
│ Country       │ Country(0.0)        │ Country(0.0)        │ Country(0.0)        │ Country(0.0)       │ Country(1.0)        │ Country(1.0)        │ Country(1.0)        │ Country(1.0)       │ Country(2.0)        │ Country(2.0)       │ Country(2.0)        │ Country(2.0)        │
├───────────────┼─────────────────────┼─────────────────────┼─────────────────────┼────────────────────┼─────────────────────┼─────────────────────┼─────────────────────┼────────────────────┼─────────────────────┼────────────────────┼─────────────────────┼─────────────────────┤
│ NoJourneys    │ NoJourneys(0.0)     │ NoJourneys(0.0)     │ NoJourneys(1.0)     │ NoJourneys(1.0)    │ NoJourneys(0.0)     │ NoJourneys(0.0)     │ NoJourneys(1.0

In [11]:
# Save the long-awaited painfully-computed preciously-regenerated full completed data.
full_completed_data.replace(num_to_cat).to_csv("full_completed_data.csv")
full_completed_data = pd.DataFrame.from_csv("full_completed_data.csv").replace(cat_to_num)

## Structure Learning

In [92]:
scorer = K2Score(test_data)
print(scorer.score(model))

hc = HillClimbSearch(completed_data)
best_model = hc.estimate()

print(scorer.score(best_model))


-8851.050358468436
-8739.714029743163


## Final Model

In [93]:
best_model.edges()

[('DangerLvl', 'AvgSpeed'),
 ('DangerLvl', 'NoAccidents'),
 ('NoAccidents', 'NoFatalities'),
 ('NoJourneys', 'Weather'),
 ('NoJourneys', 'NoAccidents'),
 ('NoJourneys', 'Season'),
 ('PoliceActivity', 'AvgSpeed'),
 ('RoadCond', 'AvgSpeed'),
 ('RoadCond', 'DangerLvl'),
 ('Season', 'RoadCond'),
 ('Weather', 'RoadCond'),
 ('Weather', 'Season')]

In [36]:
# Traing the final model on the completed data
edges = [
    ('AvgSpeed', 'PoliceActivity'),
    ('AvgSpeed', 'DangerLvl'),
    ('DangerLvl', 'NoAccidents'),
    ('DangerLvl', 'NoFatalities'),
    ('NoAccidents', 'NoFatalities'),
    ('Weather', 'NoJourneys'),
    ('NoJourneys', 'NoAccidents'),
    ('Season', 'NoJourneys'),
    ('RoadCond', 'AvgSpeed'),
    ('RoadCond', 'DangerLvl'),
    ('Season', 'RoadCond'),
    ('Weather', 'RoadCond'),
    ('Season', 'Weather'),
    ('Weekend', 'NoJourneys')
]
final_model = BayesianModel(edges)
final_model.add_node("Country")

In [37]:
final_model.fit(full_completed_data)

In [38]:
from pgmpy.readwrite.BIF import BIFWriter
writer = BIFWriter(final_model)
writer.write_bif('najmami2.bif')

## Jensen-Shannon Divergence and Total Variation Distance

In [82]:
from itertools import product

domain = tuple(list(d.values())for _, d in cat_to_num.items())
combinations = list(product(*domain))
full_combinations_index = pd.MultiIndex.from_tuples(combinations)

In [83]:
sampler = BayesianModelSampling(final_model)
model_samples = sampler.forward_sample(len(data))
model_samples_frequencies = model_samples.groupby(nodes).size()
model_samples_frequencies = model_samples_frequencies.reindex(full_combinations_index, fill_value=0)

In [89]:
full_completed_data_frequencies = full_completed_data.groupby(nodes).size().reindex(full_combinations_index, fill_value=0)
print(JSD(model_samples_frequencies.values, full_completed_data_frequencies.values))
print(total_variation_distance(model_samples_frequencies.values, full_completed_data_frequencies.values))

0.108180479577
0.00565511315316
